<a href="https://colab.research.google.com/github/befriends7/AndNewsTest/blob/master/FireAlarmiot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import io
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving smoke_detection.csv to smoke_detection.csv


In [ ]:
df_Data = pd.read_csv(io.BytesIO(uploaded['smoke_detection.csv']))

In [ ]:
df_Data.head(4)

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0


In [ ]:
df_Data.isnull().sum()

Unnamed: 0        0
UTC               0
Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5             0
NC1.0             0
NC2.5             0
CNT               0
Fire Alarm        0
dtype: int64

In [ ]:
X = df_Data[['Temperature[C]','Humidity[%]']].values

In [ ]:
X.max()

75.2

In [ ]:
X_scaled = X/75.2

In [ ]:
X_scaled[4]

array([0.26674202, 0.72726064])

In [ ]:
y = df_Data[['Fire Alarm']].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=30,random_state=42)   

In [ ]:
X_train.max()

1.0

In [ ]:
X_train.shape

(62600, 2)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=30,batch_size=10)

Epoch 1/30
6260/6260 [==============================] - 9s 1ms/step - loss: 0.5001 - accuracy: 0.7890
Epoch 2/30
6260/6260 [==============================] - 10s 2ms/step - loss: 0.4613 - accuracy: 0.8297
Epoch 3/30
6260/6260 [==============================] - 9s 2ms/step - loss: 0.4315 - accuracy: 0.8369
Epoch 4/30
6260/6260 [==============================] - 9s 2ms/step - loss: 0.4087 - accuracy: 0.8489
Epoch 5/30
6260/6260 [==============================] - 9s 1ms/step - loss: 0.3949 - accuracy: 0.8541
Epoch 6/30
6260/6260 [==============================] - 9s 1ms/step - loss: 0.3841 - accuracy: 0.8549
Epoch 7/30
6260/6260 [==============================] - 9s 1ms/step - loss: 0.3743 - accuracy: 0.8569
Epoch 8/30
6260/6260 [==============================] - 10s 2ms/step - loss: 0.3660 - accuracy: 0.8579
Epoch 9/30
6260/6260 [==============================] - 9s 1ms/step - loss: 0.3605 - accuracy: 0.8572
Epoch 10/30
6260/6260 [==============================] - 9s 1ms/step - loss: 0.3

In [ ]:
test_Data = np.array()

In [ ]:
X_test = np.array([10.084/75.2,43.02/75.2])

In [ ]:
X_test = X_test.reshape(1,2)

In [ ]:
model.predict(X_test)

1/1 [==============================] - 0s 17ms/step


array([[0.00020964]], dtype=float32)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tflite_Model = converter.convert()

In [ ]:
from pathlib import Path

In [ ]:
tflite_Model_path = Path('fireModel.tflite')
tflite_Model_path.write_bytes(tflite_Model)

3384

In [ ]:
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [ ]:
c_model_name = 'fireModel'

with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_Model, c_model_name))

In [ ]:
!apt-get update && apt-get -qq install xxd

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [102 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/g

In [ ]:
!xxd -i /content/fireModel.tflite > fireModel.cc